In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# 导入GUI基础模块，用于创建主窗口
from tkinter import Tk
# 导入文件对话框模块，用于弹出文件选择窗口
from tkinter.filedialog import askopenfilename

# 创建 Tk 实例
root = Tk()
# 隐藏主窗口
root.withdraw()
# 设置窗口始终置顶
root.attributes("-topmost", True)

# 弹出文件选择对话框
file_path = askopenfilename(parent=root, title="选择血糖值数据文件", filetypes=[("Excel files", "*.xlsx")])

# 恢复窗口的正常状态（可选）
root.attributes("-topmost", False)

# 检查用户是否选择了文件
if file_path:
    df = pd.read_excel(file_path)
else:
    print("未选择文件，程序退出。")
    root.destroy()
    exit()


In [19]:
df

,序号,时间,血糖值
0,0,2025-02-19 21:57,8.8
1,1,2025-02-19 22:00,9.0
2,2,2025-02-19 22:03,9.1
3,3,2025-02-19 22:06,9.1
4,4,2025-02-19 22:09,9.0
...,...,...,...
6716,6716,2025-03-05 21:45,6.9
6717,6717,2025-03-05 21:48,7.1
6718,6718,2025-03-05 21:51,7.1
6719,6719,2025-03-05 21:54,7.0


In [20]:
df["时间"] = pd.to_datetime(df["时间"])

In [21]:
# ---------------------- 新增字体配置 ----------------------
# 方法一：直接指定系统字体（Windows示例）
# plt.rcParams['font.family'] = 'Microsoft YaHei'  
# 方法二：直接指定系统字体（Mac示例）
# plt.rcParams['font.family'] = 'Arial Unicode MS'
# 方法三：根据操作系统选择字体
import platform

if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Microsoft YaHei'
elif platform.system() == 'Darwin':  # macOS 系统
    plt.rcParams['font.family'] = 'Arial Unicode MS'
else:
    plt.rcParams['font.family'] = 'Arial'

plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
# --------------------------------------------------------

In [22]:
# 按日期分组
# 从DataFrame df的“时间”列中提取日期信息，并获取所有唯一的日期
dates = df["时间"].dt.date.unique()

In [23]:
# 设置正常血糖范围
NORMAL_MIN = 3.9
NORMAL_MID = 7.8
NORMAL_MAX = 10.0

## 生成每日血糖折曲线图，并保存为图片放入“血糖曲线图片”文件夹中

In [24]:
import os

# 创建保存图片的文件夹
output_folder = "血糖曲线图片"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 生成每日图表
from tkinter import Label, HORIZONTAL
from tkinter import ttk
# 创建进度窗口
progress_root = Tk()
progress_root.title("生成进度")
progress_root.geometry("300x100+500+300")  # 可拖动位置
progress_root.attributes("-topmost", False)  # 取消置顶以允许自由拖动

# 进度组件
progress_label = Label(progress_root, text="准备生成图表...")  # 现在Label已定义
progress_label.pack(pady=5)
progress = ttk.Progressbar(progress_root, orient=HORIZONTAL, length=250, mode='determinate')
progress.pack()
progress_root.update()  # 初始渲染

try:
    total_dates = len(dates)
    # 生成每日图表
    for i, date in enumerate(dates):
        daily_data = df[df["时间"].dt.date == date]
        
        # 更新进度
        progress_label.config(text=f"正在生成 {date} 的图表 ({i+1}/{total_dates})")
        progress['value'] = (i+1)/total_dates*100
        progress_root.update()  # 保持窗口响应
        
        # ... 原有的图表生成代码不变 ...
        plt.figure(figsize=(12, 4))
        plt.plot(daily_data["时间"], daily_data["血糖值"], marker="o", linestyle="-", color="b", markersize=3)
        plt.axhspan(NORMAL_MIN, NORMAL_MID, alpha=0.2, color="green", label="正常范围")
        plt.axhspan(NORMAL_MID, NORMAL_MAX, alpha=0.2, color="yellow", label="控制范围")
        plt.title(f"血糖值变化曲线 - {date}")
        plt.xlabel("时间")
        plt.ylabel("血糖值 (mmol/L)")
        plt.xticks(rotation=45)
        plt.grid(True, linestyle="--", alpha=0.7)
        plt.legend()
    
        plt.tight_layout()
    # 修改保存路径，将图片保存到指定文件夹
        plt.savefig(os.path.join(output_folder, f"血糖曲线_{date}.png"), dpi=300, bbox_inches="tight")
        plt.close()  # 关闭图表避免内存泄漏
        
finally:
    progress_root.destroy()  # 完成后关闭进度窗口
    root.destroy()

In [25]:
from docx import Document
from docx.shared import Inches
from docx.enum.section import WD_ORIENT
from docx.shared import Cm

# 创建一个新的Word文档
doc = Document()

# 修改页面方向为横向
section = doc.sections[0]
new_width, new_height = section.page_height, section.page_width
section.orientation = WD_ORIENT.LANDSCAPE
section.page_width = new_width
section.page_height = new_height

# 设置页边距，将上下边距调整为 0.8 厘米，左右边距仍为 1.27 厘米
section.left_margin = Cm(1.27)
section.right_margin = Cm(1.27)
section.top_margin = Cm(0.8)
section.bottom_margin = Cm(0.8)

# 定义图片文件夹路径
output_folder = "血糖曲线图片"

# 获取图片文件夹中的所有图片文件
import os
image_files = [f for f in os.listdir(output_folder) if f.endswith('.png')]

# 按日期排序图片文件名（假设文件名中包含日期信息）
image_files.sort()

# 每页添加两张图片
for i in range(0, len(image_files), 2):
    # 处理第一张图片
    image_file1 = image_files[i]
    image_path1 = os.path.join(output_folder, image_file1)
    # 添加图片标题，使用文件名作为标题
    doc.add_heading(image_file1.replace('.png', ''), level=1)
    # 添加图片到文档，设置图片宽度为页面宽度减去一些边距
    paragraph = doc.add_paragraph()
    run = paragraph.add_run()
    run.add_picture(image_path1, width=Inches(new_width.inches - 1))

    # 检查是否有第二张图片
    if i + 1 < len(image_files):
        image_file2 = image_files[i + 1]
        image_path2 = os.path.join(output_folder, image_file2)
        # 添加图片标题，使用文件名作为标题
        doc.add_heading(image_file2.replace('.png', ''), level=1)
        # 添加图片到文档，设置图片宽度为页面宽度减去一些边距
        paragraph = doc.add_paragraph()
        run = paragraph.add_run()
        run.add_picture(image_path2, width=Inches(new_width.inches - 1))

    # 只有在不是最后一组图片且已经添加了两张图片时才添加分页符
    if i + 2 < len(image_files):
        doc.add_page_break()

# 保存Word文档
doc.save('血糖曲线报告.docx')

print("血糖曲线图片已整理到 '血糖曲线报告.docx' 文件中。")

血糖曲线图片已整理到 '血糖曲线报告.docx' 文件中。
